In [ ]:
!pip install pfhedge

### Setup

In [ ]:
import pfhedge
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
from pfhedge.instruments import BrownianStock
from pfhedge.instruments import EuropeanOption
from pfhedge.nn import BlackScholes

stock = BrownianStock()
derivative = EuropeanOption(stock, strike=1.0)
m = BlackScholes(derivative)

In [ ]:
derivative.simulate(2)
lm = derivative.log_moneyness()
t = derivative.time_to_maturity()
v = derivative.ul().volatility

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = torch.stack(data, dim=-1)

    def __len__(self):
        return self.data.size(1)

    def __getitem__(self, index):
        return self.data[:, index, :].unsqueeze(1)

In [ ]:
ds = MyDataset([lm, t, v])

In [ ]:
m

BSEuropeanOption(strike=1.)

In [ ]:
m.delta()

tensor([[0.5113, 0.4944, 0.5653, 0.5905, 0.5212, 0.5374, 0.6688, 0.6958, 0.5537,
         0.4483, 0.5362, 0.5158, 0.6702, 0.7202, 0.7738, 0.7170, 0.6043, 0.6803,
         0.3114, 0.4061, 1.0000],
        [0.5113, 0.6327, 0.8310, 0.8234, 0.6675, 0.7780, 0.8617, 0.9027, 0.7882,
         0.7577, 0.7229, 0.8444, 0.8902, 0.9153, 0.9712, 0.9729, 0.9405, 0.8407,
         0.9715, 1.0000, 1.0000]])

In [ ]:
m.delta(lm, t, v)

tensor([[0.5113, 0.4944, 0.5653, 0.5905, 0.5212, 0.5374, 0.6688, 0.6958, 0.5537,
         0.4483, 0.5362, 0.5158, 0.6702, 0.7202, 0.7738, 0.7170, 0.6043, 0.6803,
         0.3114, 0.4061, 1.0000],
        [0.5113, 0.6327, 0.8310, 0.8234, 0.6675, 0.7780, 0.8617, 0.9027, 0.7882,
         0.7577, 0.7229, 0.8444, 0.8902, 0.9153, 0.9712, 0.9729, 0.9405, 0.8407,
         0.9715, 1.0000, 1.0000]])

In [ ]:
for i in ds:
    print(m(i).size())

torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])
torch.Size([2, 1, 1])


### Shape (or size) is very important!!!

In [ ]:
pfhedge.nn.functional.pl(stock.spot, m.delta())

tensor(0.0117)

### pfhedge.nn.modules.hedger.compute_hedge

In [ ]:
from pfhedge.instruments import BaseInstrument, BaseDerivative
from torch import Tensor
from typing import Optional
from typing import List

def compute_hedge(
        self, derivative: BaseDerivative, hedge: Optional[List[BaseInstrument]] = None
    ) -> Tensor:

        inputs = self.inputs.of(derivative, self)
        hedge = self._get_hedge(derivative, hedge)

        # Check that the spot prices of the hedges have the same sizes
        if not all(h.spot.size() == hedge[0].spot.size() for h in hedge):
            raise ValueError("The spot prices of the hedges must have the same size")

        (n_paths, n_steps), n_hedges = hedge[0].spot.size(), len(hedge)
        if inputs.is_state_dependent():
            zeros = hedge[0].spot.new_zeros((n_paths, 1, n_hedges))
            save_prev_output(self, input=(), output=zeros)
            outputs = []
            for time_step in range(n_steps - 1):
                input = inputs.get(time_step)  # (N, T=1, F)
                outputs.append(self(input))  # (N, T=1, H)
            outputs.append(outputs[-1])
            output = torch.cat(outputs, dim=-2)  # (N, T, H)
        else:
            # If all features are state-independent, compute the output at all
            # time steps at once, which would be faster.
            input = inputs.get(None)  # (N, T, F)
            output = self(input)  # (N, T, H)
            # This maintains consistency with the previous implementations.
            # In previous implementation the loop is computed for 0...T-2 and
            # the last time step is not included.
            output[..., -1, :] = output[..., -2, :]

        output = output.transpose(-1, -2)  # (N, H, T)

        return output


In [ ]:
from pfhedge.nn import Hedger
hedger = Hedger()

In [ ]:
hedger.__call__()

In [ ]:
torch.nn.Module

In [ ]:
hedger.forward()

In [ ]:
for i in ds:
    print(i.size())

torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])
torch.Size([2, 1, 3])


In [ ]:
outputs = []
for i in ds:
    outputs.append(m(i))

outputs = torch.cat(outputs, dim=-2)
# outputs = outputs.transpose(-1,-2)
outputs

tensor([[[0.5113],
         [0.4944],
         [0.5653],
         [0.5905],
         [0.5212],
         [0.5374],
         [0.6688],
         [0.6958],
         [0.5537],
         [0.4483],
         [0.5362],
         [0.5158],
         [0.6702],
         [0.7202],
         [0.7738],
         [0.7170],
         [0.6043],
         [0.6803],
         [0.3114],
         [0.4061],
         [1.0000]],

        [[0.5113],
         [0.6327],
         [0.8310],
         [0.8234],
         [0.6675],
         [0.7780],
         [0.8617],
         [0.9027],
         [0.7882],
         [0.7577],
         [0.7229],
         [0.8444],
         [0.8902],
         [0.9153],
         [0.9712],
         [0.9729],
         [0.9405],
         [0.8407],
         [0.9715],
         [1.0000],
         [1.0000]]])

In [ ]:
outputs = []
for i in ds:
    outputs.append(m(i))

outputs = torch.cat(outputs, dim=-1)
outputs

tensor([[[0.5113, 0.4944, 0.5653, 0.5905, 0.5212, 0.5374, 0.6688, 0.6958,
          0.5537, 0.4483, 0.5362, 0.5158, 0.6702, 0.7202, 0.7738, 0.7170,
          0.6043, 0.6803, 0.3114, 0.4061, 1.0000]],

        [[0.5113, 0.6327, 0.8310, 0.8234, 0.6675, 0.7780, 0.8617, 0.9027,
          0.7882, 0.7577, 0.7229, 0.8444, 0.8902, 0.9153, 0.9712, 0.9729,
          0.9405, 0.8407, 0.9715, 1.0000, 1.0000]]])

### check

In [ ]:
from pfhedge.instruments import BrownianStock
from pfhedge.instruments import EuropeanOption
from pfhedge.nn import BlackScholes

In [ ]:
_ = torch.manual_seed(42)

In [ ]:
derivative = EuropeanOption(BrownianStock(), maturity=5/250)

In [ ]:
derivative.simulate(2)

In [ ]:
derivative.ul().spot

tensor([[1.0000, 1.0016, 1.0044, 1.0073, 0.9930, 0.9906],
        [1.0000, 0.9919, 0.9976, 1.0009, 1.0076, 1.0179]])

In [ ]:
model = BlackScholes(derivative)

In [ ]:
from pfhedge.nn import Hedger

In [ ]:
hedger = Hedger(model, model.inputs())

In [ ]:
hedger.compute_hedge(derivative).squeeze(1)

tensor([[0.5056, 0.5295, 0.5845, 0.6610, 0.2918, 0.2918],
        [0.5056, 0.3785, 0.4609, 0.5239, 0.7281, 0.7281]])

In [ ]:
hedger.compute_portfolio(derivative)

tensor([-0.0062,  0.0106])

### Our version

In [ ]:
lm = derivative.log_moneyness()
t = derivative.time_to_maturity()
v = derivative.ul().volatility

ds = MyDataset([lm, t, v])

In [ ]:
outputs = []
for i in ds:
    outputs.append(m(i))

outputs = torch.cat(outputs, dim=-1)
outputs.squeeze(1)

tensor([[0.5056, 0.5295, 0.5845, 0.6610, 0.2918, 0.0000],
        [0.5056, 0.3785, 0.4609, 0.5239, 0.7281, 1.0000]])

In [ ]:
spot = torch.stack([derivative.ul().spot], dim=1)

In [ ]:
spot.size()

torch.Size([2, 1, 6])

In [ ]:
pfhedge.nn.functional.pl(spot, outputs)

tensor([-0.0062,  0.0106])

### Final step

In [ ]:
derivative.payoff()

tensor([0.0000, 0.0179])

In [ ]:
pfhedge.nn.functional.pl(spot, outputs, payoff=derivative.payoff())

tensor([-0.0062, -0.0073])

In [ ]:
hedger.compute_pl(derivative)

tensor([-0.0062, -0.0073])

In [ ]:
hedger.price(derivative, n_paths=2)

tensor(0.0112)

In [ ]:
hedger.criterion.cash(hedger.compute_portfolio(derivative), derivative.payoff())

# For the next step

In [ ]:
def compute_hedge(model, ds):
    outputs = []
    for i in ds:
        outputs.append(model(i))

    return torch.cat(outputs, dim=-1)

In [ ]:
def compute_portfolio(model, ds, derivative):

    spot = torch.stack([derivative.ul().spot], dim=1)
    unit = compute_hedge(model, ds)

    return pfhedge.nn.functional.pl(spot, unit)

In [ ]:
def compute_portfolio_2(model, ds, derivative):

    spot = torch.stack([derivative.ul().spot], dim=1)
    unit = compute_hedge(model, ds)

    return pfhedge.nn.functional.pl(spot, unit, payoff=derivative.payoff())